## 파일 불러오기

In [1]:
import pandas as pd
import glob

files = glob.glob('*.csv')
files

['pre_201901.csv',
 'pre_201902.csv',
 'pre_201903.csv',
 'pre_201904.csv',
 'pre_201905.csv',
 'pre_201906.csv',
 'pre_201907.csv',
 'pre_201908.csv',
 'pre_201909.csv',
 'pre_201910.csv',
 'pre_201911.csv',
 'pre_201912.csv']

## Text Crawl 이전 데이터 전처리용

## 양 끝 공백제거, 경향신문 조선일보 제거, 중복 제거
## 중복, 예외 태그 제거
## 메인 카테고리 문화, IT_과학, 지역 제거
## 부고, 날씨, 오늘의, 시사일본어 중국어, 한겨레 알림, e글 중심 제거

In [8]:
category_droplist = ['문화', 'IT', '지역']
headline_droplist = ['날씨', '<부고>', '부고 -', '궂긴', '오늘의', '시사일본어', '시사중국어', '일 알림', 'e글중심']

for file in files:
    df = pd.read_csv(file, encoding="utf-8-sig")
    df = df.dropna(axis=0)
    df = df.apply(lambda x: x.str.strip(), axis = 1) # 모든 데이터 양 끝 공백 제거

    df = df[df.press != "경향신문"] # 경향신문 데이터 제거
    df = df[df.press != "조선일보"] # 조선일보 데이터 제거
    df = df.drop_duplicates(["url"]) # 중복 제거
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # 중복, 예외 태그 제거
        if df['headline'][index][-2:] == '중복':
            df['headline'][index] = df['headline'][index][0:len(df['headline'][index])-3].strip()
            continue
        if df['headline'][index][-2:] == '예외':
            df['headline'][index] = None
    
    for index in range(0, len(df)): # 문화, IT, 지역 제거
        if df['category'][index][0:2] in category_droplist:
            df['category'][index] = None
            continue
    
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # 부고, 날씨, 오늘의, 시사일본어 중국어, 한겨레 알림, e글 중심 제거
        if '날씨' in df['category'][index]:
            df['category'][index] = None
            continue
        for drop in headline_droplist:
            if drop in df['headline'][index]:
                df['headline'][index] = None
                break
                
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    for index in range(0, len(df)): # len 50미만 컷
        if len(df['text'][index]) < 50:
            df['text'][index] = None
            
    df = df.dropna(axis=0)
    df = df.reset_index(drop=True)
    
    df.to_csv("pre_"+file, encoding="utf-8-sig", mode = 'w') # 저장시 파일 이름 주의

## Text Crawl 이후 본문 전처리용 

## 본문 내용 일정 길이 안 넘어가면 지워버리기

In [9]:
for file in files:
    df = pd.read_csv(file, encoding='utf-8-sig')
    for index in range(0, len(df)):
        if len(df['text'][index]) < 50:
            df['text'][index] = None
    df = df.dropna(axis=0)
    #df.to_csv(file, encoding="utf-8-sig", mode = 'w') # 저장시 파일 이름 주의   


In [2]:
import pandas as pd
df = pd.read_csv(files[0], encoding='utf-8-sig')
df['press'].value_counts()

중앙일보    3950
세계일보    3334
국민일보    3221
동아일보    3161
서울신문    2874
한겨레     1985
문화일보    1360
내일신문    1331
Name: press, dtype: int64

In [6]:
category_droplist = ['문화', 'IT', '지역']
headline_droplist = ['날씨', '<부고>', '부고 -', '궂긴', '오늘의', '시사일본어', '시사중국어', '일 알림', 'e글중심']


df = pd.read_csv(files[0], encoding="utf-8-sig")
df = df.dropna(axis=0)
df = df.apply(lambda x: x.str.strip(), axis = 1) # 모든 데이터 양 끝 공백 제거

df = df[df.press != "경향신문"] # 경향신문 데이터 제거
df = df[df.press != "조선일보"] # 조선일보 데이터 제거
df = df.drop_duplicates(["url"]) # 중복 제거
df = df.reset_index(drop=True)

for index in range(0, len(df)): # 문화, IT, 지역 제거
    print(df['headline'][index])
    if df['category'][index][0:2] in category_droplist:
        df['category'][index] = None
        continue

df = df.dropna(axis=0)
df = df.reset_index(drop=True)

for index in range(0, len(df)): # 부고, 날씨, 오늘의, 시사일본어 중국어, 한겨레 알림, e글 중심 제거
    if '날씨' in df['category'][index]:
        df['category'][index] = None
        continue
    for drop in headline_droplist:
        if drop in df['headline'][index]:
            df['headline'][index] = None
            break
            
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

for index in range(0, len(df)): # len 50미만 컷
    if len(df['text'][index]) < 50:
        df['text'][index] = None
        
df = df.dropna(axis=0)
df = df.reset_index(drop=True)

df.to_csv("test_" + files[0], encoding="utf-8-sig", mode = 'w') # 저장시 파일 이름 주의

문희상 “임시정부 100년…새 역사 시작하는 새해 될 것”
새해에도 암울한 대구·경북 지역경제
경남 승강기안전공단 지역과 상생 동반성장 활동
문 대통령 신년사 “한 분 한 분의 삶 나아지도록 노력하겠다”
이마트, 올해 상반기 ‘한우 등심’ 초저가로 가격 동결
새해 달라지는 지역 정책…고교 무상급식·무상교복 전국 확산
“그 비서 몸매 좋던데”…국회 페미 “페미니즘=시대정신, 국회부터 바뀌어야”
공무원 업무추진비 사용, 휴일·심야 증빙자료 내야
현대·기아·르노·한국지엠 8개 신차 실내공기질 '합격'
새해 세계경제 운명 가를 5대 변곡점
3·1운동 떠오르는 사람은? 82%가 “유관순”
차세대 유니콘, 나야 나!
“관심과 사랑을 이웃에…” 푸른전력 김한상 대표 전북 54번째 아너소사이어티
여성 임원 약진…‘철옹성’ 금융권 유리천장에도 실금
[사설]경제현장은 벼랑끝인데 與는 “지표 나아지고 있다” 낙관론
육아휴직 급여 오른다…새해부터 통상임금 50%
[사설]“靑이 적자국채 발행 압력” 주장… 덮으려 말고 진위 명확히 가려라
[사설]좌절한 20대에게 꿈과 희망 주는 새해로
제약업계, 화학약품 위주서 ‘바이오’로 급속 전환 [4차 산업혁명 - 신년특집]
프랜차이즈 갑질 못하게…본사가 파는 재료값 공개 확대
고양·파주 “경의선 늘려달라” 한 목소리
“착한 패딩” 홍보해놓고…손님 오면 “그건 싸구려”
미니스톱 인수 장기화…여러 추측 난무
국민들, 3·1운동 핵심정신 1순위로 “친일 청산” 꼽았다
주식보다 채권-金 투자해볼만… 안개 걷히는 하반기 노려라
“통화정책 완화 유지…기업 구조조정 통해 생산성 높여야”
경제회복·실업해소…새해 국민 소망 두 가지
보험 들기 전 설계사 정보 인터넷서 살펴보세요
금리 들썩… 변동형 → 고정형 대출 갈아타기 고려를
“한반도 평화 담는 제2독립선언서 필요” 47%
한-일 관계 시급한 과제는 1020 “일본군 위안부” 3040 “친일 잔재 청산”
국토부 “조종사 채용계획 내라”… 항공사 “인력확보 못해 불이익”
제주개발공사, 국민임대주택사업 첫 시

손혜원 의원이 신재민 자살 소동 직후 삭제한 글
[더 골프숍] 혼마의 홍보대사 김종필과 저스틴 로즈
"계획적 범죄" vs "심신미약"… 의사 살인 피의자 감형될까
경기도교육청, 북한 이해와 공감대 확산에 나섰다
폭풍에 컨테이너 와르르···독성물질 든 3개가 사라졌다
손혜원, 너무 부지런해서 탈… 與 하락세 멈춘 순간 '아차차'
"우리 아이 그 학교 못 보낸다" 교육청 마비시킨 강남 부자들
‘새해 첫 잭팟’ 美복권 4700억원 당첨자 나왔다… 뉴욕에서 판매
“나도 무서웠다” 임세원 교수 살해범 엄마가 전한 아들의 ‘폭력성’
한국에만 있는 '빠른년생' 문화… 꼭 있어야할까
“수영 중 전화?좀 대단하다” 투신사망 여성이 들은 말(영상)
신재민, 일반 병실서 이틀째 치료 중
청와대 임종석 후임 노영민 주중대사 유력
‘액체괴물’ 알고보니 붕소 괴물…“반복 노출 땐 생식·발달 유해”
신재민 고려대 동문들이 민변에 사과한 이유
‘중국서도 안 팔린다’ 애플 주가 10% 폭락
2018년 어려웠다지만…새해에도 뜨거운 '프랜차이즈 창업' 관심
청와대 국민청원에 ‘디스패치 폐간’이 화제인 이유… 청원 16건 등장
靑 임종석 후임에 노영민 유력…3선 의원 출신 ‘文 대통령 최측근’
초등생에 칼 휘둘러도...정신질환자 관리 못하는 정신건강복지법
"3억이 6개월새 5억"...더욱 무서워진 '재건축 부담금 공포'
신재민 부모 “아들 극단적 선택으로 물의 일으켜 죄송…많이 괴로워했다”
표창원, ‘신재민 폭로’ 입장 밝힌 김동연에 “깊고 따뜻한 마음에 공감”
청소년들의 살벌한 돈거래…안 갚는다고 때리고, 집 찾아가 행패
“선배 용기에 감명받았다” 신재민 응원한 고대 동문들(영상)
‘딸 정신 이상으로 힘들어’ 양천구 아파트서 숨진 모녀
문근영, 급성구획증후군 ‘완치’ 후 예능으로 복귀
꼬막, 문어, 굴…외식업계 2019 제철 식재료로 건강한 입맛 잡는다
소득 낮고 시골 살수록…당뇨병·고혈압·비만 발병률↑
낸시 펠로시, 8년만에 미국 하원의장 선출
[편집장 편지] 축하해 주세요~~
배규식 노동연

영진전문대학교, 삼성 KB DB금융 등 취업 러시
구미대학교 기계자동차공학부, 현지 연수 통해 일본취업 시동
구미상공회의소 신년회 ‘경제위기 극복하자’ 한 목소리
구미직업전문학교, 일반고 학생 9명 국가자격증 취득
성폭행 주장 자살한 30대 부부, 1·2심 뒤집고 가해자 유죄
‘말레이판 윈저공?’…왕좌 비우고 ‘결혼 휴가’ 떠난 국왕 중도 퇴위에 설 분분
[단독]檢 ‘징용 소송 전합 회부 계획 전달은 양승태 지시’ 진술 확보
‘윤창호법 적용’ 배우 손승원 “후배가 운전” 거짓말도
'전두환은 광주지법에 출석하라'
한국서 바둑 유학한 10세 소녀, 일본 최연소 기사 됐다
반민정 측 “조덕제 2차가해에 정신적으로 무너진 상태…법적 조치 검토”
정부, 최저임금 결정 구조 이원화하기로
지난 해에만 45명, 6년간 공무원 102명 배출한 서정대…비결은 특화교육
[김지석 칼럼] 개성공단·금강산관광 재개부터
유은혜 부총리 “고교학점제 만족도 높아…지속 확대할 것”
檢 "'MBC 장악' 김재철·원세훈, 표현의 자유 심각하게 침해"
2018년 소비자 트렌드로 살펴본 국내 이커머스 시장
5·18 재단·단체 “전두환, 용서 구할 마지막 기회 놓치지 말라”
정지선 없는 교차로서 황색 신호등으로 바뀌면…갈까? 말까?
이미 대선주자급…유시민은 정계복귀설을 왜 계속 부인하나?
길이 4.2m 산갈치가 강원 해변에···고성 주민이 발견
남미 속 중국…수리남, 공식 언어로 ‘중국어’ 채택
글로벌 유아용품 브랜드 페도라, ‘2019 홍콩 유아용품박람회’ 참가
전두환 자택 앞 집결 보수단체 “강제구인? 우리 밟고 해”…지만원도 참가
영진전문대 해외취업반 현지 연수받는다
“‘애플 쇼크’ 어쩌나…” 국내 IT업계는 ‘고심중’
식물인간 여성 출산…요양병원 남성직원 성폭행 수사
최저임금 결정체계 30년만에 개편…"전문가가 구간 설정"
국민은행 파업 D-1, 고객에 안내문…“인터넷·모바일뱅킹, 8일 이용 가능”
코레일, 횟수차감형 할인카드 ‘KTX N카드’ 1만매 돌파
DGIST, 비정규직 행정·기술직 정

정동영 “북미회담지, 베트남 가능성…양국에 정치적 함의 커”
양 前 대법원장, 476일 만의 대법원 방문 성사될까?
조명래 장관, 주한 일본대사와 한-일 환경협력방안 논의
청와대 홍보라인 채운 기자출신 3인방, 호흡 잘 맞을까
"'너 정도면 얼굴 괜찮다'도 '언어 성폭력'"
[현장에서] 똑같은 QLED, 똑같은 즉석카메라…한국산이면 베끼고 보는 중국 기업
서울교육청 “학생들이 선생님한테 ‘~님’ ‘~쌤’ 부르는 거 아냐” … 진땀 해명
[서소문사진관] 포토샵이 부른 참사?...호주 총리 사진에 왼발만 둘
배고파 가구 갉을때까지 방치됐다가 처참하게 죽은 강아지…견주 고발당해
규제 막혔던 '콜버스'(전세버스 공유 서비스), 이젠 달릴 수 있다
20대 노동자 잇따라 참변…김용균씨 등 한달새 3명 숨져
‘가격도 세금도 네 맘대로’, 단 그 가격에 팔아야 한다면…
퇴임 뒤에도 ‘제왕’?…대법원에서 입장 밝히겠다는 ‘피의자 양승태’
조재범 “성폭행 안 해… 심석희가 지목한 범죄 장소가 그 근거”
美 식물인간 여성 출산 충격…경찰, 병원 남직원 전원 DNA 채취
"치실 잘못 쓰면 발암성 화합물 PFAS 노출 위험"
정두언 "내 시간을 팝니다, 분당 2만원에~" 30분 이상 구매자엔 노래까지
정무비서관 복기왕, 국정홍보비서관 여현호…"향후 '친문' 대거 입성 가능성"
“독점기업을 해체하라”…우파의 ‘자본주의 구하기’는 성공할까
미국, 전쟁 지속이냐 종식이냐…트럼프발 논란 확대
극우 논객 지만원, 나경원 집 찾아가 비난 세례 [사진in세상]
[속보] 베이징 미·중 무역협상 마무리 …중국 "좋은 결과"
북경반점 점심 먹고 제약공장 시찰…김정은 방중 마치고 열차 귀국
문 대통령에 유시민까지 가짜뉴스 총공세…'노무현 정부 시즌2' 되나
전자발찌 절단 50대·음란사이트 운영자 국내 송환
여수 양계장 화재… 2500㎡ 전소
[편집국에서] ‘두명의 폼페이오’와 2차 북-미 정상회담 / 이용인
[왜냐면] 유가족의 품격보다 사회의 품격이 필요하다 / 이승홍
[왜냐면] 국회 ‘촛불입법연대'로

“편의점 도시락, 전자레인지에 돌려 드세요”…세균 99% 감소
'양심없는' 병역 거부자 색출위해 검찰이 낸 아이디어
“연쇄도미노”…시골에서도 최저임금 ‘아우성’
"경제·민생 최우선" vs "현실도피"…文 신년 기자회견 놓고 엇갈린 여야
‘사법 농단 정점’ 양승태 전 대법원장 어떤 입장 내놓나
희망과 감동을 전했던 미담의 소름 돋는 반전…‘천사의사’ 김씨의 이중생활
[속보] 파인텍 노사, 고공농성 426일 만에 교섭 타결
[기고]해외여행 2870만명 시대 ‘시차 장애 예방법’ 알고 계시나요
“머리 짧은 여자라 맞았다” 피해의식에 선동된 ‘女론’ [페미 논란]
파월 Fed 의장, "올해 두차례 금리인상은 호경기 전제"
노랑 조끼 시위대가 과속 단속 카메라의 60% 망가뜨리는 이유
트럼프, 다보스포럼 참석 전격 취소
[CES 2019] 자율주행시대 대비하는 완성차 업계…‘차 안에서 답을 찾다’
불안한 최저임금 결정체계 개편안 "노사 갈등 줄이려다 되레 키울 수도"
“호주 거주 탈북민, 마약 관련 범죄로 추방위기…항소한 상태”
이해찬 “김정은, 2월 북미정상회담 이어 서울 답방 전망”
양승태, 오늘 검찰 출석…사법부 수장에서 피의자로 전락
김예령 기자 “문 대통령을 걱정하는 마음으로 한 질문”
하와이 왕복 58만7300원 특가
中소학교에 ‘꿈의 도서실’선물
양승태, 오늘(11일) 검찰 출석…대국민 입장 발표 예정
굴뚝농성 426일 만에…파인텍 노사 협상 극적 타결
'누가 불렀나' 코치설에 靑과 與 "육참총장 콜이 팩트, 코치설은 오보"· 野 "따져보자"[이슈톡톡]
직장인, 설에 41만4000원 지출…세뱃돈 18만1000원 준비
통영 남쪽 공해에서 낚싯배 전복…3명 사망, 2명 실종
[영상] 김예령 기자 ‘근자감’ 질문에 동료들의 반응은?(ft.손석희·최경영)
양승태, 오늘 검찰 출석…오전 8시부터 시위대 서초동 집결
배종찬 “유시민 눈빛 보면…불출마 의지 ‘활활’”
시위대 집결…양승태 출석 앞둔 대법원 앞 긴장 고조
올해도 불꽃 튈 직장인 '생존 경쟁'…소망 1위 '

...

‘정치 심의’ 비판받던 방심위, ‘오늘밤 김제동’ 문제없음 결론
외통위 “국민 납득할 협정 타결해야” 한미 방위비 협상 촉구 결의문 발표
삼성전자, 지난해 미국서 35억원 로비자금 사용…김석한 변호사 소속 로펌도 이용
안산에서 또 홍역 확진…시흥·안산 홍역 환자 10명으로 늘어
유아 탑승차량 2중 추돌사고 높아, 카시트 안전성 살피는게 중요
임종석 전 靑비서실장, UAE특임 외교특보…한병도, 이라크 특보 위촉
유시민 “대선주자 여론 조사에서 내 이름 빼줘” 여론조사심의위에 공문
안락사 논란 케어 박소연 대표 “도살 없으면 안락사도 없어” 잔혹 영상 SNS에
보쉬, 사물인터넷 기반 ‘미래의 공장’ 제안
미 셧다운 한 달…‘생활고’ 공무원들 TV 들고 전당포 행렬
[박찬수 칼럼] 총리 출신이 대통령 될 수 없는 이유
마케도니아 국호 변경에 6만 시민 운집..“‘마케도니아’ 아예 쓰지 말아야”
도둑은 토요일 새벽에 열린 창문으로 들어온다
“모험자본 공급 늘리자” 개인 전문투자자 문턱 낮춘다
경북 초중교 23곳, 입학생 한 명 못 받았다
“강릉 펜션사고 재발 막자”…사고 백서 나온다
[사설] 정부, 중재 노력 배가해 ‘한반도 대전환’ 이끌어야
'선거법 위반 혐의' 법정 출석한 원희룡 제주지사
박원순 “김정은에 서울역·한강 소개할 것… 찬반 시위 문제 없어”
나이들면 찾아오는 '야간뇨'…한림대강남성심병원 23일 무료강좌
한국당, 2·27 전당대회 세부일정 확정…본격 전대 모드 돌입
文 “기초연금 지자체 부담률 개선 검토”…부산 북구청장 재정파탄 읍소에 응답
겨울철 가장 짧은 건조시간 입증한 ‘LG 트롬 건조기’, 그 비결은?
檢, 손혜원 '목포 의혹' 수사 착수…다른 고발 사건들 병합 검토
[유레카] 김정은 위원장의 생일 / 박병수
구청장 “복지 부담” 호소 전화에 문 대통령 “함께 고민해 봅시다”
한국전기안전공사 대구경북지부, ‘에버’로 정전 복구
도어락 비밀번호 알아내 가정집 절도 일삼은 30대 男 법정구속
‘1기 靑 비서진’ 임종석·한병도 외교특보 위촉
“트럼프,

연준 자산, 얼마나 줄어들까
"건강한 5G 산업 생태계 만들자"
미국, 한국제품 견제 심해졌다
지난해 석유제품 수출량 사상최대
[자유한국당 2·27 전당대회] 너도나도 출마 … 당대표 예비후보만 10여명
문 대통령, 오늘 전략회의 주재
청와대 '공직기강' 강화, 내부 폭로 영향?
본질 벗어난 '손혜원 공방' 점입가경
전대 1차 관문은 컷오프 … 3명? 5명?
거대 양당 어깃장에 휴지조각된 '선거법 논의 합의문'
재취업률 낮은데 실업급여는 부실
정부·지자체 '안전·위기대응' 다짐
베이비부머 떠난 지자체 인사난맥상
대통령 세종집무실 요구 높아
은평 성모병원에서 일하실 분?
폼페이오 "스웨덴협상서 더 진전"
미 대법원 심리유보로 불법체류청년 70만명 보호 연장
미측 무리한 요구, 방위비 협상 '헛바퀴'
홍남기 "대규모 유휴 국유지 개발"
한은, 올해 경제성장률 전망치 조정하나
평균노동소득 43세에 '정점' 찍은뒤 하락
현대해상, '설연휴 차량 안전운행 지원서비스'
NH농협금융, 디지털 추진 결의대회
<여론마당미국은 대승적 차원에서 방위분담금 협상에 임해야
“대기국장은 자리 걸고 미세먼지 대책 세워라”
<美國에서 본 한반도文, 실리콘밸리 기술 戰場 와 보라
대학총장들 “반값등록금 발목잡혀 재정악화 심각”
체육관련 100개 기관 대상… 여가부 폭력예방교육 점검
檢, 김태우 자택 압수수색… 소환 임박
손학규 “손혜원, 대통령 부인 권위로 국회의원 된 것 아닌가”
“양승태 구속하라”vs“풀어줘라”…법원 밖도 ‘전쟁터’
[기고] 지금 ‘안락사 논쟁’을 할 때인가
中 군사굴기 견제하는 美·日…'무기 전시장' 된 아태 지역 [디펜스 포커스]
靑 "영부인 친구라도 민정수석실 국회의원 감찰 못해"
삼성 브랜드 가치 103조원… 세계 5위
'프로기각러' 비난받은 허경호···박병대 영장판사 누구
故김용균씨 어머니 “같은 비극 없게 나라서 바로세워주길”
국민연금 ‘月200만원’ 수령자 10명 나왔다…100만원 이상 ‘20만명’ 돌파
수은의 습격…수산물 내장, 덩치 큰 생선 조심하세요
생

[친절한 쿡캐스터] 서울 영하 13도 체감온도… 감기 조심하세요!
부산 동래경찰서, 불법 전단지 인쇄업자·대부업자 41명 검거
주택 상속받은 2주택자, 세금폭탄 피하는 방법은
의사 머리 치고, 간호사 눈 찔러…응급실 난동 하루에 3번 꼴
납치돼 성노예로 팔려간 여성만 2만명… 칼 빼든 나이지리아
"열달 동안 생리 안하게 해줄까?" 선생님은 늑대였습니다
“중국 공기 안 좋아” 라디오스타 발언, 공식 사과한 황치열
1년치 끊은 헬스장이 갑자기 문 닫아, 환불 가능할까요?
"불나면 어쩌려고"…'흉물'로 방치된 한강편의점 '들여다보니' [김기자의 현장+]
“아빠 사형시켜달라” 청원 낸 딸, 30년 선고에 한 말(영상)
“체력 약한 여경 필요 없다” 남성 경찰도 절레절레
미 연방정부 셧다운 시한부 해소…트럼프 “3주간 재가동”
브라질 남동부서 광산 댐 3개 붕괴…200여명 실종된듯
국립한국방송통신대 프라임칼리지, 1월 25일~30일 신·편입생 추가모집 진행
27년 시설 생활을 끝냈을 때 저, 하상윤을 찾았어요
폼페이오, 베네수엘라 특사에 '강성 매파' 에이브럼스 임명
“나는 꿈꾼다, ‘꿀알바’의 나라를…”
옷에서 자동차까지…소유에서 공유, 공유에서 구독으로
주유소서 돌연 옷 벗고 사진찍은 여성, 이유가…
“개·고양이 도살 잔혹사 당장 금지하라”
글로링크에듀센터 인천송도캠퍼스, 신입생 입학 설명회 진행
한국은 바다, 태국은 매장 점령···'검은 반도체' 김의 전쟁
"84학번에 우병우였다면, 85학번은 안태근이었다"
"美앞바다로 열도 옮기고싶다"···한일관계, 日간부의 속내
설 차례상 비용, 전통시장 ‘25만4215원’ 대형유통업체 ‘34만9941원’
'구관이 명관'…수십년째 각광받는 무기들은 [박수찬의 軍]
양승태, 구치소에서 맞은 71번째 생일…미역국 전날 배식
마케도니아→‘북마케도니아’ 국호 변경…그리스와 갈등 해소
미국 국가비상사태 ‘일단 스톱’ …셧다운 일시 해제 법안 가결
황치열 "中 눈도 못뜰 만큼 공기 안좋아" 비난 여론 높아지자 "죄송하다"
‘손혜원 지역구

“청주도 미세먼지 심각… ‘대기관리권역’ 지정돼야”
‘강남 클럽 폭행’ 피해자 주장 20대, 경찰 자빠뜨리고 욕설
구제역 확진 하루만에 안성서 구제역 또 발생
[오늘의시선] 최악 치닫는 한·일관계, 이대론 안된다
[데스크의눈] 규제완화, 보수만의 옷 아니다
[WT논평] American energy independence at home
“너무 귀여워” 연해주서 아기 백두산 호랑이 포착
승리, '버닝썬' 폭행 사건 당일 클럽서 포착돼
파키스탄 대법원, ‘무함마드 부정적 발언’ 아시아 비비 무죄 확정
日방위상 "한국과 냉각기 필요…상황되면 고위급 대화 모색"
30일 동정
동탄 살인 용의자, 경찰 붙잡히는 과정서 숨져
'케이트 vs 메건' 도 넘은 팬 싸움…英왕실 SNS회사에 지원요청
국민연금 수탁자위 "대한항공·한진칼 경영참여 재논의 안해"
안성서 또 구제역 확진…인근 농장으로 확산 우려
광주여대 특수교사 임용시험서 강세
조선대 언어치료학과 국가고시 합격률 95.5%
연세대 '대학 창업생태계 조성' MOU 체결
중등 실력이 고등 내신, 수능을 좌우한다
승리가 운영하는 '버닝썬'은 어떤 클럽?…"헤미넴이 1억원짜리 술마신 곳"
금속노조 대구지부 “노조 불법도청 책임자 구속 수사하라”
중앙대 평생교육원 ‘주중학사과정’ 신입생 모집
건강하고 빛나는 피부를 위한 단 하나의 선택
靑경제보좌관의 남 탓, 차별 발언···"국민에 할 소리냐"
명절 풍습 바뀌어도…10가구 중 6가구는 설 쇤다
국민·기초연금 합쳐도 ‘저소득·단기가입자' 빈곤 탈피 어려워
‘적폐 청산’ 노력…한국 국가청렴도 여섯 계단 상승해 45위
저항 안해서 무죄? “부하 성폭행한 해군 간부들에게 상식적 판결을”
한국당 선관위 “황교안, 오세훈 전당대회 출마할 수 있다”
잇몸 관리·충치 예방까지… 3중 케어 기능성 치약
전원책 "손석희는 동승자 누군지 투명하게 밝혀야… 털면 다 털린다"
홈쇼핑서 매진 행진… 주부들의 워너비 주방가전
“더 즐겁고 편리하게” 쇼핑·문화 등 간편 결제
사무금융노조 “정규직화 위해 임금 

자치경찰제 실효성 논란 속, 제주 137명 인력 추가 파견
얼굴에 침 뱉고 물컵 던지고… 공소장에 담긴 이명희 갑질
조재범 항소심, 1심보다 무거운 징역 1년6개월
SK㈜, 스마트글라스 시장 진출 美 키네스트랄社에 1억달러 투자
예상밖 판결에 도민들 충격… “도정 동력 떨어질라” 우려
美 골든타임때문에… 도쿄올림픽 육상 결승 오전에 열릴 가능성
홍남기 부총리 “가업상속공제 문턱 낮아지고 증권거래세 인하 방안도 검토”
KB국민은행, 간편뱅킹 앱 ‘리브’ 인기
38년 동안 찾았는데… 3살 때 잃어버린 아들 극적 상봉
대림산업 김상우, 부회장 승진
[단독] ‘용산참사 과잉 진압 부인’ 김석기 의원 징계안 제출
학교 밖 청소년들 지원 강화… 공교육 수준의 학습권 보장
[김치로드] 나트륨 폭탄? 김치에 관한 오해와 진실
[Vision 2019] “미래를 선점하라” … 인공지능·자율주행 등 첨단 기술 개발 ‘올인’
[2019 대한민국 브랜드 명예의전당] 소비자들이 선택한 최고의 브랜드 한자리에
[Vision 2019] 주력 사업 주도권 키우고 성장엔진 육성 박차
[Vision 2019] 올해 신차 13종 출시, 글로벌 시장 수익성 강화
[Vision 2019] 혁신·내실 통한 지속적 성장 기반 구축 주력
[2019 대한민국 브랜드 명예의전당] 브랜드는 산업의 경쟁력이자 국가 경제 발전의 초석
[2019 대한민국 브랜드 명예의전당] 혁신적인 디지털 전략으로 고객과 직원에 집중
[Vision 2019] M&A, 선택과 집중 바탕 새 사업 진출 모색
[2019 대한민국 브랜드 명예의전당] 가입 고객 370만 돌파한 대표 모바일 플랫폼
[2019 대한민국 브랜드 명예의전당] 매물 검색부터 금융까지 이어주는 원스톱 서비스
[2019 대한민국 브랜드 명예의전당] 퇴직연금 전담 관리 조직 출범, 체계적 고객관리
[Vision 2019] 사회적 가치 창출 통한 비즈니스 모델 혁신
[2019 대한민국 브랜드 명예의전당] 차별화된 혜택 제공으로 신용카드 시장 선도
[2019 대한민국 브랜

In [5]:
df = pd.read_csv(files[0], encoding="utf-8-sig")
df = df.dropna(axis=0)
df = df.reset_index(drop=True)
df

,date,press,category,headline,url,text
0,2019/01/01,한겨레,정치,문희상 “임시정부 100년…새 역사 시작하는 새해 될 것”,http://www.hani.co.kr/arti/politics/politics_g...,"문희상 국회의장이 2019년 새해를 “대한민국 임시정부 100년, 임시의정원 100..."
1,2019/01/01,세계일보,"지역,경제",새해에도 암울한 대구·경북 지역경제,http://www.segye.com/content/html/2018/12/31/2...,내년도 대구와 경북의 경제성장률이 1%대에 그칠 것이란 암울한 전망이 나왔다.31일...
2,2019/01/01,세계일보,"경제,지역",경남 승강기안전공단 지역과 상생 동반성장 활동,http://www.segye.com/content/html/2018/12/31/2...,경남 한국승강기안전공단이 지역과 상생하며 동반성장하는 다양한 활동에 나서고 있다.공...
3,2019/01/01,중앙일보,정치,문 대통령 신년사 “한 분 한 분의 삶 나아지도록 노력하겠다”,http://news.joins.com/article/olink/22844371,"문재인 대통령은 1일 “이 겨울, 더 따뜻하게 세상을 밝히라는 촛불의 마음 결코 잊..."
4,2019/01/01,동아일보,경제,"이마트, 올해 상반기 ‘한우 등심’ 초저가로 가격 동결",http://news.donga.com/3/all/20181231/93504466/3,이마트가 올해 상반기 동안 한우 등심 가격을 인하·동결한다고 1일 밝혔다. 이마트는...
...,...,...,...,...,...,...
24402,2019/01/31,중앙일보,"국제,정치","비핵화 협상 기로…뭉치는 북방3각, 멀어지는 남방3각",https://news.joins.com/article/olink/22934944,2014년 3월26일 네덜란드 헤이그 미국대사관. 버락 오바마 대통령이 박근혜 대통...
24403,2019/01/31,중앙일보,"국제,정치",일본서 여성의원·활동가에 '강매 괴롭힘'…수법보니,https://news.joins.com/article/olink/22934950,"일본에서 주장이 강한 여성 정치인이나 시민단체 활동가 등을 겨냥한 협박, 괴롭힘 사..."
24404,2019/01/31,국민일보,"사회,지역",숨진 아내 몸에 ‘방어흔’… 이틀 만에 발견된 노부부의 비극,http://news.kmib.co.kr/article/view.asp?arcid=...,기사 내용과 무관. 게티이미지뱅크전북 고창의 한 아파트에서 80대 노부부가 숨진 채...
24405,2019/01/31,서울신문,"국제,정치","“트럼프·시진핑, 북미정상회담 후 2월 말 정상회담 개최 논의”",http://www.seoul.co.kr/news/newsView.php?id=20...,"美언론 “류허, 中하이난서 정상회담 제안” 도널드 트럼프 미국 대통령과 시진핑 중국..."
